In [11]:
DATA_IN = "../data/raw/rees46/_/"
DATA_OUT = "../data/raw/rees46/filtered/"

import findspark
findspark.init()

In [2]:
import pyspark 
from pyspark.sql import SparkSession

# later on - probably add dtypes in load phase
from pyspark.sql.types import StringType, IntegerType,\
    DoubleType, StructType, StructField, DateType

In [3]:
sc = pyspark.SparkContext()
conf = pyspark.SparkConf().setAll([('spark.executor.memory', '10g'), ('spark.executor.cores', '4'),
    ('spark.cores.max', '4'), ('spark.driver.memory','8g'), ("spark.kryoserializer.buffer.max","1g"),
    ("spark.sql.execution.arrow.pyspark.enabled", "true")])
sc.stop()
sc = pyspark.SparkContext(conf=conf)

ss = SparkSession.builder.getOrCreate()
# terminate # set parames # get or create
ss.sparkContext.setLogLevel("ERROR")
ss.sparkContext.getConf().getAll()

22/04/19 11:27:39 WARN Utils: Your hostname, mf-H110M-S2H resolves to a loopback address: 127.0.1.1; using 192.168.0.108 instead (on interface enx00d2b176ce73)
22/04/19 11:27:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/04/19 11:27:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


[('spark.driver.port', '33653'),
 ('spark.app.startTime', '1650360462281'),
 ('spark.executor.memory', '10g'),
 ('spark.executor.id', 'driver'),
 ('spark.app.name', 'pyspark-shell'),
 ('spark.executor.cores', '4'),
 ('spark.cores.max', '4'),
 ('spark.kryoserializer.buffer.max', '1g'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.memory', '8g'),
 ('spark.driver.host', '192.168.0.108'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.id', 'local-1650360462342'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.sql.execution.arrow.pyspark.enabled', 'true')]

In [4]:
def get_sdf_info(sdf, name=""):
    rc = sdf.count(); cc = len(sdf.columns) # note - wrap this into a fuc
    print("The {} dataset has shape of ({},{}), and following dtypes\n{}".\
        format(name, rc, cc, sdf.dtypes))

In [5]:
import glob
for f in glob.glob(DATA_IN+"*.gz"):
    if "events" not in locals():
        events = ss.read.csv(f,header=True, inferSchema=True, nanValue="null")
    else:
        events.union(ss.read.csv(f,header=True, inferSchema=True, nanValue="null"))
#events = ss.read.csv(glob.glob(DATA_DIR+"*.gz")[2],header=True, inferSchema=True)
get_sdf_info(events)
events.show(3)

The  dataset has shape of (67542878,9), and following dtypes
[('event_time', 'string'), ('event_type', 'string'), ('product_id', 'int'), ('category_id', 'bigint'), ('category_code', 'string'), ('brand', 'string'), ('price', 'double'), ('user_id', 'int'), ('user_session', 'string')]
+--------------------+----------+----------+-------------------+--------------------+-----+-------+---------+--------------------+
|          event_time|event_type|product_id|        category_id|       category_code|brand|  price|  user_id|        user_session|
+--------------------+----------+----------+-------------------+--------------------+-----+-------+---------+--------------------+
|2019-12-01 00:00:...|      view|   1005105|2232732093077520756|construction.tool...|apple|1302.48|556695836|ca5eefc5-11f9-450...|
|2019-12-01 00:00:...|      view|  22700068|2232732091643068746|                null|force| 102.96|577702456|de33debe-c7bf-44e...|
|2019-12-01 00:00:...|      view|   2402273|223273210076987446

In [6]:
# keep interactions for users with >=10 trans
from pyspark.sql.functions import col, countDistinct
users_to_keep = events.filter(col("event_type")=="purchase").\
    groupBy("user_id").agg(countDistinct("user_session").\
        alias("n_transactions")).filter(col("n_transactions")>=10).select(col("user_id"))
events = events.join(users_to_keep, ["user_id"], "inner")
get_sdf_info(events)

The  dataset has shape of (1806685,9), and following dtypes
[('user_id', 'int'), ('event_time', 'string'), ('event_type', 'string'), ('product_id', 'int'), ('category_id', 'bigint'), ('category_code', 'string'), ('brand', 'string'), ('price', 'double'), ('user_session', 'string')]


In [7]:
# carry-out date conversion
from pyspark.sql.functions import col,to_timestamp
events = events.withColumn("event_time", to_timestamp(col("event_time")))
get_sdf_info(events)
events.show(3)

The  dataset has shape of (1806685,9), and following dtypes
[('user_id', 'int'), ('event_time', 'timestamp'), ('event_type', 'string'), ('product_id', 'int'), ('category_id', 'bigint'), ('category_code', 'string'), ('brand', 'string'), ('price', 'double'), ('user_session', 'string')]


+---------+-------------------+----------+----------+-------------------+--------------------+-------+------+--------------------+
|  user_id|         event_time|event_type|product_id|        category_id|       category_code|  brand| price|        user_session|
+---------+-------------------+----------+----------+-------------------+--------------------+-------+------+--------------------+
|512415830|2019-12-17 11:43:24|      view|  26300742|2053013554725912943|appliances.kitche...|lucente|320.73|70641837-0ad0-41c...|
|512415830|2019-12-17 11:43:54|      cart|  26300742|2053013554725912943|appliances.kitche...|lucente|320.73|70641837-0ad0-41c...|
|512415830|2019-12-17 11:44:10|  purchase|  26300742|2053013554725912943|appliances.kitche...|lucente|320.73|70641837-0ad0-41c...|
+---------+-------------------+----------+----------+-------------------+--------------------+-------+------+--------------------+
only showing top 3 rows



In [8]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
cols_old = events.columns
cols_to_replace = ["user_id", "product_id", "category_id", "event_type"]
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index")
    for column in cols_to_replace]
sip = Pipeline(stages=indexers)
sip = sip.fit(events)
events = sip.transform(events)

In [9]:
# lookup for events
event_types = events.select(col("event_type_index").alias("event_type_id"),
    col("event_type").alias("event_type_name")).dropDuplicates()
for c in cols_to_replace:
    events = events.withColumn(c,col(c+"_index"))
events = events.withColumn("event_type_id", col("event_type"))
# carve-out tabs
products = events.select(["product_id", "category_id", "brand"]).dropDuplicates()
categories = events.select(["category_id", "category_code"]).dropDuplicates()
events = events.select(["event_time", "user_id", "product_id", "event_type_id", "price", "user_session"])
get_sdf_info(products, "products")
get_sdf_info(categories, "categories")
get_sdf_info(events, "events")

The products dataset has shape of (81586,3), and following dtypes
[('product_id', 'double'), ('category_id', 'double'), ('brand', 'string')]


The categories dataset has shape of (1065,2), and following dtypes
[('category_id', 'double'), ('category_code', 'string')]


The events dataset has shape of (1806685,6), and following dtypes
[('event_time', 'timestamp'), ('user_id', 'double'), ('product_id', 'double'), ('event_type_id', 'double'), ('price', 'double'), ('user_session', 'string')]


In [14]:
# try to save both of the datasets to gz
import os, shutil
shutil.rmtree(DATA_OUT, ignore_errors=True)
os.makedirs(DATA_OUT)
events.write.csv(DATA_OUT+"events", compression="gzip", header=True)
products.write.csv(DATA_OUT+"products", compression="gzip", header=True)
categories.write.csv(DATA_OUT+"categories", compression="gzip", header=True)

In [ ]:
# test on tree-based mapping
import pandas as pd
cat_df = ss.read.csv(DATA_OUT+"categories", header=True).toPandas()

# we might use this mapping
def get_edges(leaf_id, category_code):
    if category_code is None:
        cat=[]
    else:
        cat = category_code.split(".")
    res = []
    prev = leaf_id
    while len(cat)>0:
        curr = cat.pop()
        res.append([curr, prev])
        prev=curr
    if len(cat)==0:
        res.append([None,prev])
    return pd.DataFrame(res,
        columns=["parent_category", "category"])

tree = []
for i,r in cat_df.iterrows():
    tree.append(get_edges(r["category_id"], r["category_code"]))
pd.concat(tree)        